In [17]:
import pandas as pd

df = pd.read_csv("C:/Users/Hxtreme/Downloads/churn_dataset.csv")
df.head()
df.info()
df.describe()
df['Churn'].value_counts()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64374 entries, 0 to 64373
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   CustomerID         64374 non-null  int64  
 1   Age                63087 non-null  float64
 2   Gender             64374 non-null  object 
 3   Tenure             63087 non-null  float64
 4   Usage Frequency    64374 non-null  int64  
 5   Support Calls      64374 non-null  int64  
 6   Payment Delay      63087 non-null  float64
 7   Subscription Type  64374 non-null  object 
 8   Contract Length    64374 non-null  object 
 9   Total Spend        63087 non-null  float64
 10  Last Interaction   64374 non-null  int64  
 11  Churn              64374 non-null  int64  
dtypes: float64(4), int64(5), object(3)
memory usage: 5.9+ MB


Churn
0    33881
1    30493
Name: count, dtype: int64

In [18]:
target_counts = df['Churn'].value_counts()

print("Count of each class:")
print(target_counts)

print(f"\nNumber of 0s: {target_counts.get(0, 0)}")
print(f"Number of 1s: {target_counts.get(1, 0)}")

Count of each class:
Churn
0    33881
1    30493
Name: count, dtype: int64

Number of 0s: 33881
Number of 1s: 30493


In [19]:
df.isnull().sum()

CustomerID              0
Age                  1287
Gender                  0
Tenure               1287
Usage Frequency         0
Support Calls           0
Payment Delay        1287
Subscription Type       0
Contract Length         0
Total Spend          1287
Last Interaction        0
Churn                   0
dtype: int64

In [20]:
df['Age'].fillna(df['Age'].median(), inplace=True)
df['Tenure'].fillna(df['Tenure'].median(), inplace=True)
df['Payment Delay'].fillna(df['Payment Delay'].mean(), inplace=True)
df['Total Spend'].fillna(df['Total Spend'].median(), inplace=True)

C:\Users\Hxtreme\AppData\Local\Temp\ipykernel_12728\2680725908.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Age'].fillna(df['Age'].median(), inplace=True)
C:\Users\Hxtreme\AppData\Local\Temp\ipykernel_12728\2680725908.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

F

In [21]:
df.dtypes

CustomerID             int64
Age                  float64
Gender                object
Tenure               float64
Usage Frequency        int64
Support Calls          int64
Payment Delay        float64
Subscription Type     object
Contract Length       object
Total Spend          float64
Last Interaction       int64
Churn                  int64
dtype: object

In [22]:
# finding unique values in each column
df.nunique()

CustomerID           64374
Age                     48
Gender                   2
Tenure                  60
Usage Frequency         30
Support Calls           11
Payment Delay           32
Subscription Type        3
Contract Length          3
Total Spend            901
Last Interaction        30
Churn                    2
dtype: int64

In [23]:
df = df.drop(columns = ['CustomerID'], axis = 1)
df

,Age,Gender,Tenure,Usage Frequency,Support Calls,Payment Delay,Subscription Type,Contract Length,Total Spend,Last Interaction,Churn
0,22.0,Female,25.0,14,4,27.0,Basic,Monthly,598.0,9,1
1,41.0,Female,28.0,28,7,13.0,Standard,Monthly,584.0,20,0
2,47.0,Male,27.0,10,2,29.0,Premium,Annual,757.0,21,0
3,35.0,Male,9.0,12,5,17.0,Premium,Quarterly,232.0,18,0
4,53.0,Female,58.0,24,9,2.0,Standard,Annual,533.0,18,0
...,...,...,...,...,...,...,...,...,...,...,...
64369,45.0,Female,33.0,12,6,21.0,Basic,Quarterly,947.0,14,1
64370,37.0,Male,6.0,1,5,22.0,Standard,Annual,923.0,9,1
64371,25.0,Male,39.0,14,8,30.0,Premium,Monthly,327.0,20,1
64372,50.0,Female,18.0,19,7,22.0,Standard,Monthly,540.0,13,1


In [24]:
from sklearn.preprocessing import LabelEncoder

label_cols = ['Gender', 'Contract Length', 'Subscription Type']
le = LabelEncoder()
for col in label_cols:
    df[col] = le.fit_transform(df[col].astype(str))


In [25]:
# outlier code using iqr method
# the code is about finding the outliers
num_cols = ['Age', 'Tenure', 'Usage Frequency', 'Support Calls',
            'Payment Delay', 'Total Spend']

def remove_outliers_iqr(df, columns):
    df_clean = df.copy()
    for col in columns:
        Q1 = df_clean[col].quantile(0.25)
        Q3 = df_clean[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        df_clean = df_clean[(df_clean[col] >= lower_bound) & (df_clean[col] <= upper_bound)]

    return df_clean

# Apply outlier removal
df_no_outliers = remove_outliers_iqr(df, num_cols)
print(df_no_outliers)

        Age  Gender  Tenure  Usage Frequency  Support Calls  Payment Delay  \
0      22.0       0    25.0               14              4           27.0   
1      41.0       0    28.0               28              7           13.0   
2      47.0       1    27.0               10              2           29.0   
3      35.0       1     9.0               12              5           17.0   
4      53.0       0    58.0               24              9            2.0   
...     ...     ...     ...              ...            ...            ...   
64369  45.0       0    33.0               12              6           21.0   
64370  37.0       1     6.0                1              5           22.0   
64371  25.0       1    39.0               14              8           30.0   
64372  50.0       0    18.0               19              7           22.0   
64373  52.0       0    45.0               15              9           25.0   

       Subscription Type  Contract Length  Total Spend  Last In

In [26]:
def classify_income(row):
    if row['Total Spend'] > 1000: return 'High'
    elif row['Total Spend'] > 700: return 'Medium'
    else: return 'Low'

df['Spend'] = df.apply(classify_income, axis=1).astype('category')


In [27]:
df['Spend_zscore'] = df.groupby('Subscription Type')['Total Spend'].transform(
    lambda x: (x - x.mean()) / x.std()
)

In [28]:
df.to_csv("cleaned_churn_data.csv", index=False)